In [1]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions.{col}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.classification.{GBTClassifier, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier, NaiveBayes, LinearSVC, LogisticRegression, OneVsRest}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.Vectors
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
137,application_1576053140655_0138,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.sql.functions.col
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{GBTClassifier, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier, NaiveBayes, LinearSVC, LogisticRegression, OneVsRest}
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.linalg.Vectors
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar


In [2]:
val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("datasets/creditcard.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw: org.apache.spark.sql.DataFrame = [Time: string, V1: string ... 29 more fields]


In [3]:
// cast all the column to Double type.
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 29 more fields]


In [4]:
// convert the label from {0, 1} to {1, 2}
//val labelConverter = new FuncTransformer(udf {d: Double => if (d==0) 2 else d }).setInputCol("Class").setOutputCol("Class")
val labelConverter = new StringIndexer().setInputCol("Class").setOutputCol("label")
val assembler = new VectorAssembler().setInputCols(Array("V3", "V4", "V9", "V10", "V11", "V12", "V14", "V16", "V17", "V18","V19")).setOutputCol("assembled")
val scaler = new MinMaxScaler().setInputCol("assembled").setOutputCol("features")
//val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
println("Generate feature from raw data:")
data.select("features", "label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.StringIndexer = strIdx_b52075e99e08
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_82e91ac2d7d7
scaler: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_86f810437633
pipeline: org.apache.spark.ml.Pipeline = pipeline_4aeb023e06e5
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_4aeb023e06e5
data: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 32 more fields]
Generate feature from raw data:
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.88136490328633...|  0.0|
|[0.84029849039390...|  0.0|
|[0.86814081926190...|  0.0|
|[0.86848364774806...|  0.0|
|[0.86425070140685...|  0.0|
|[0.85718742663956...|  0.0|
|[0.83819983804773...|  0.0|
|[0.85603110871019...|  0.0|
|[0.83545216735689...|  0.0|
|[0.85551101187934...|  0.0|
|[0.85324951634142...|  0.0|
|[0.82226325870284...|  0.0|
|[0.84406658980857...|  0.0|
|[0.85177230541965...|  0.0|
|[0.86586281889129

### step 2. split the dataset into training and validation dataset.

In [5]:
val data0 = data.filter($"Class" === "0").cache()
val data1 = data.filter($"Class" === "1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
data1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [6]:
val splitTime0 = data0.stat.approxQuantile("Time", Array(0.7), 0.001).head
val splitTime1 = data1.stat.approxQuantile("Time", Array(0.2), 0.001).head
val trainingData0 = data0.filter(s"Time<$splitTime0").cache()
val validationData0 = data0.filter(s"Time>=$splitTime0").cache()
val trainingData1 = data1.filter(s"Time<$splitTime1").cache()
val validationData1 = data1.filter(s"Time>=$splitTime1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitTime0: Double = 132882.0
splitTime1: Double = 35926.0
trainingData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
trainingData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [7]:
trainingData0.select("V1", "Class").show(10)
trainingData1.select("V1", "Class").show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----+
|                V1|Class|
+------------------+-----+
|  -1.3598071336738|  0.0|
|  1.19185711131486|  0.0|
| -1.35835406159823|  0.0|
|-0.966271711572087|  0.0|
| -1.15823309349523|  0.0|
|-0.425965884412454|  0.0|
|  1.22965763450793|  0.0|
|-0.644269442348146|  0.0|
| -0.89428608220282|  0.0|
| -0.33826175242575|  0.0|
+------------------+-----+
only showing top 10 rows

+-------------------+-----+
|                 V1|Class|
+-------------------+-----+
|   -2.3122265423263|  1.0|
|   -3.0435406239976|  1.0|
|  -2.30334956758553|  1.0|
|  -4.39797444171999|  1.0|
|   1.23423504613468|  1.0|
|0.00843036489558254|  1.0|
| 0.0267792264491516|  1.0|
|  0.329594333318222|  1.0|
|  0.316459000444982|  1.0|
|  0.725645739819857|  1.0|
+-------------------+-----+
only showing top 10 rows



In [8]:
val time0 = trainingData0.select("Time")
val time1 = trainingData1.select("Time")
time0.describe("Time").show()
time1.describe("Time").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

time0: org.apache.spark.sql.DataFrame = [Time: double]
time1: org.apache.spark.sql.DataFrame = [Time: double]
+-------+-----------------+
|summary|             Time|
+-------+-----------------+
|  count|           198889|
|   mean|70433.87985258109|
| stddev|34483.90351654142|
|    min|              0.0|
|    max|         132881.0|
+-------+-----------------+

+-------+------------------+
|summary|              Time|
+-------+------------------+
|  count|                98|
|   mean|19695.571428571428|
| stddev| 9823.899477725412|
|    min|             406.0|
|    max|           35906.0|
+-------+------------------+



In [9]:
val time0v = validationData0.select("Time")
val time1v = validationData1.select("Time")
time0v.describe("Time").show()
time1v.describe("Time").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

time0v: org.apache.spark.sql.DataFrame = [Time: double]
time1v: org.apache.spark.sql.DataFrame = [Time: double]
+-------+------------------+
|summary|              Time|
+-------+------------------+
|  count|             85426|
|   mean| 151656.3990471285|
| stddev|10987.155988334218|
|    min|          132882.0|
|    max|          172792.0|
+-------+------------------+

+-------+-----------------+
|summary|             Time|
+-------+-----------------+
|  count|              394|
|   mean|95932.13959390864|
| stddev|40919.16913390304|
|    min|          35926.0|
|    max|         170348.0|
+-------+-----------------+



In [10]:
val trainingData = trainingData0.unionAll(trainingData1)
val validationData = validationData0.unionAll(validationData1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [11]:
println(" Training set statistics: 1 represents fraud and 0 represents normal")
trainingData.groupBy("Class").count().show()
println(" validation set statistics: 1 represents fraud and 0 represents normal")
validationData.groupBy("Class").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Training set statistics: 1 represents fraud and 0 represents normal
+-----+------+
|Class| count|
+-----+------+
|  0.0|198889|
|  1.0|    98|
+-----+------+

 validation set statistics: 1 represents fraud and 0 represents normal
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  394|
+-----+-----+



In [12]:
//Gradient-boosted tree classifier
// Train a GBT model.
//setImpurity("entropy") 
val t = System.nanoTime
val gbt = new GBTClassifier().setLabelCol("label").setFeaturesCol("features").setMaxIter(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 634824721671153
gbt: org.apache.spark.ml.classification.GBTClassifier = gbtc_543701f0be9d


In [13]:
val modelGBT = gbt.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelGBT: org.apache.spark.ml.classification.GBTClassificationModel = GBTClassificationModel (uid=gbtc_543701f0be9d) with 10 trees
durationtrain: Double = 37.349055926

initial model training finished.
Training process takes 37.349055926 secs


In [14]:
val s = System.nanoTime
val predictionsGBT = modelGBT.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsGBT.cache()
//val predictionsAndLabel: RDD[Row] = df.rdd= predictionsGBT.select("prediction", "label")
predictionsGBT.select("prediction", "label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 634862920934467
predictionsGBT: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.233884456

initial model training finished.
Training process takes 0.233884456 secs
res24: predictionsGBT.type = [V1: double, V2: double ... 35 more fields]
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [15]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsGBT.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  394|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85628|
|       1.0|  192|
+----------+-----+



In [16]:
println(s"Matrice de confusion :")
predictionsGBT.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85417|
|       1.0|  0.0|    9|
|       0.0|  1.0|  211|
|       1.0|  1.0|  183|
+----------+-----+-----+



In [17]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsGBT)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsGBT))
println("Accuracy = " + evaluator1.evaluate(predictionsGBT))
println("Precision = " + evaluator2.evaluate(predictionsGBT))
println("Recall = " + evaluator3.evaluate(predictionsGBT))
println("F1 = " + evaluator4.evaluate(predictionsGBT))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_14a3dbed4e70
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_f61224d91df8
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_0408d33ddfa6
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_067e420355d3
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_e13d9c3d4a29
accuracy: Double = 0.9974364949895129
Area Under ROC Curve = 0.7321808253671863
Accuracy = 0.9974364949895129
Precision = 0.9973319623367839
Recall = 0.9974364949895129
F1 = 0.9969961758536711
Test Error = 0.002563505010487077


In [18]:
// Random forest classifier
// Train a RandomForest model.
val rf = new RandomForestClassifier().setLabelCol("label").setFeaturesCol("features").setNumTrees(10)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_eaa6a0810c4b
t: Long = 634878088317937


In [19]:
val modelRF = rf.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelRF: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_eaa6a0810c4b) with 10 trees
durationtrain: Double = 7.285872074

initial model training finished.
Training process takes 7.285872074 secs


In [20]:
val s = System.nanoTime
val predictionsRF = modelRF.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsRF.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 634885984064761
predictionsRF: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.220985071

initial model training finished.
Training process takes 0.220985071 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [21]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsRF.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  394|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85605|
|       1.0|  215|
+----------+-----+



In [22]:
println(s"Matrice de confusion :")
predictionsRF.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85425|
|       1.0|  0.0|    1|
|       0.0|  1.0|  180|
|       1.0|  1.0|  214|
+----------+-----+-----+



In [23]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsRF)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsRF))
println("Accuracy = " + evaluator1.evaluate(predictionsRF))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_066ac2ec6ec6
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_be44b81f5bc4
accuracy: Double = 0.9978909345140993
Area Under ROC Curve = 0.7715677510419264
Accuracy = 0.9978909345140993
Test Error = 0.0021090654859007296


In [24]:
// Decision Tree classifier
// Train a DecisionTree model.
val dt = new DecisionTreeClassifier().setLabelCol("label").setFeaturesCol("features")
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_7bbd8d7a9b2b
t: Long = 634896125982293


In [25]:
val modelDT = dt.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelDT: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_7bbd8d7a9b2b) of depth 5 with 19 nodes
durationtrain: Double = 3.740223603

initial model training finished.
Training process takes 3.740223603 secs


In [26]:
val s = System.nanoTime
val predictionsDT = modelDT.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsDT.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 634902015708853
predictionsDT: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.213650972

initial model training finished.
Training process takes 0.213650972 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [27]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsDT.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  394|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85634|
|       1.0|  186|
+----------+-----+



In [28]:
println(s"Matrice de confusion :")
predictionsDT.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85424|
|       1.0|  0.0|    2|
|       0.0|  1.0|  210|
|       1.0|  1.0|  184|
+----------+-----+-----+



In [29]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsDT)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsDT))
println("Accuracy = " + evaluator1.evaluate(predictionsDT))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_d5962a985626
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_885b3fcc053a
accuracy: Double = 0.9975297133535307
Area Under ROC Curve = 0.7334908320330916
Accuracy = 0.9975297133535307
Test Error = 0.0024702866464693107


In [30]:
// SVM classifier
// Train a SVM model.
val svm = new LinearSVC().setLabelCol("label").setFeaturesCol("features").setMaxIter(100).setRegParam(0.1) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

svm: org.apache.spark.ml.classification.LinearSVC = linearsvc_ea3bdf244cae
t: Long = 634910124795369


In [31]:
val modelSVM = svm.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelSVM: org.apache.spark.ml.classification.LinearSVCModel = linearsvc_ea3bdf244cae
durationtrain: Double = 48.22468486

initial model training finished.
Training process takes 48.22468486 secs


In [32]:
val s = System.nanoTime
val predictionsSVM = modelSVM.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsSVM.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 634960737026960
predictionsSVM: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationprediction: Double = 0.21152174

initial model training finished.
Training process takes 0.21152174 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [33]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsSVM.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  394|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85820|
+----------+-----+



In [34]:
println(s"Matrice de confusion :")
predictionsSVM.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85426|
|       0.0|  1.0|  394|
+----------+-----+-----+



In [35]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsSVM)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsSVM))
println("Accuracy = " + evaluator1.evaluate(predictionsSVM))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_08bb8af8c391
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_2b9071cccc2c
accuracy: Double = 0.9954089955721277
Area Under ROC Curve = 0.5
Accuracy = 0.9954089955721277
Test Error = 0.004591004427872303


In [36]:
// Logistic Regression classifier
val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setTol(1E-6).setFitIntercept(true) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_2b7735b0be5b
t: Long = 634968885588303


In [37]:
val Modellr = lr.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Modellr: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_2b7735b0be5b, numClasses = 2, numFeatures = 11
durationtrain: Double = 5.521060701

initial model training finished.
Training process takes 5.521060701 secs


In [38]:
val s = System.nanoTime
val predictionsLR = Modellr.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsLR.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 634976796879101
predictionsLR: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.261064914

initial model training finished.
Training process takes 0.261064914 secs
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|[0.83519927540838...|
|       0.0|  0.0|[0.81537651325625...|
|       0.0|  0.0|[0.80942908533559...|
|       0.0|  0.0|[0.82934646499618...|
|       0.0|  0.0|[0.84448182135514...|
+----------+-----+--------------------+
only showing top 5 rows



In [39]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsLR.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  394|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85730|
|       1.0|   90|
+----------+-----+



In [40]:
println(s"Matrice de confusion :")
predictionsLR.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85426|
|       0.0|  1.0|  304|
|       1.0|  1.0|   90|
+----------+-----+-----+



In [41]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsLR)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsLR))
println("Accuracy = " + evaluator1.evaluate(predictionsLR))
println("Precision = " + evaluator2.evaluate(predictionsLR))
println("Recall = " + evaluator3.evaluate(predictionsLR))
println("F1 = " + evaluator4.evaluate(predictionsLR))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_19655dc502f5
evaluator2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_25c7900f0f31
evaluator3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_2e18945e63ee
evaluator4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_e8f5f75623a5
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_d64ed42fdbb9
accuracy: Double = 0.996457702167327
Area Under ROC Curve = 0.6142131979695431
Accuracy = 0.996457702167327
Precision = 0.9964702632141149
Recall = 0.996457702167327
F1 = 0.9953483908360317
Test Error = 0.0035422978326730137


In [42]:
// Naive Bayes classifier
// Train a NaiveBayes model.
val nb = new NaiveBayes().setLabelCol("label").setFeaturesCol("features").setSmoothing(1.0)
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

nb: org.apache.spark.ml.classification.NaiveBayes = nb_ad8f33c6d9c7
t: Long = 634988931189986


In [43]:
/*val scaler = new MinMaxScaler().setInputCol("features").setOutputCol("featuresScaled")
val pipeline = new Pipeline().setStages(Array(scaler))
val pipelineModel1 = pipeline.fit(trainingData)
val pipelineModel2 = pipeline.fit(trainingData)
val datatrain = pipelineModel1.transform(trainingData)
val datatest = pipelineModel2.transform(validationData)*/

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
val modelNB = nb.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelNB: org.apache.spark.ml.classification.NaiveBayesModel = NaiveBayesModel (uid=nb_ad8f33c6d9c7) with 2 classes
durationtrain: Double = 1.56790144

initial model training finished.
Training process takes 1.56790144 secs


In [45]:
val s = System.nanoTime
val predictionsNB = modelNB.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsNB.select("prediction", "label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 634992627785381
predictionsNB: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.248656706

initial model training finished.
Training process takes 0.248656706 secs
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [46]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsNB.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  394|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85820|
+----------+-----+



In [47]:
println(s"Matrice de confusion :")
predictionsNB.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85426|
|       0.0|  1.0|  394|
+----------+-----+-----+



In [48]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsNB)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsNB))
println("Accuracy = " + evaluator1.evaluate(predictionsNB))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_82213f028ff3
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_c8795cbad360
accuracy: Double = 0.9954089955721277
Area Under ROC Curve = 0.5
Accuracy = 0.9954089955721277
Test Error = 0.004591004427872303


In [49]:
// Multilayer Perceptron Classifier
// create the trainer and set its parameters
val t = System.nanoTime
val layers = Array[Int] (11, 6, 3, 2)
val mlp = new MultilayerPerceptronClassifier().setLayers(layers).setLabelCol("label").setFeaturesCol("features").setTol(1E-4).setBlockSize(128).setSeed(1234L).setMaxIter(25) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 635001270876544
layers: Array[Int] = Array(11, 6, 3, 2)
mlp: org.apache.spark.ml.classification.MultilayerPerceptronClassifier = mlpc_f3dae351c706


In [50]:
// Train a Multilayer Perceptron model.
val modelMLP = mlp.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

modelMLP: org.apache.spark.ml.classification.MultilayerPerceptronClassificationModel = mlpc_f3dae351c706
durationtrain: Double = 17.340589886

initial model training finished.
Training process takes 17.340589886 secs


In [51]:
val s = System.nanoTime
val predictionsMLP = modelMLP.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsMLP.select("prediction","label").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s: Long = 635019446739934
predictionsMLP: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 35 more fields]
durationprediction: Double = 0.205073942

initial model training finished.
Training process takes 0.205073942 secs
+----------+-----+
|prediction|label|
+----------+-----+
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
|       0.0|  0.0|
+----------+-----+
only showing top 5 rows



In [52]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsMLP.groupBy("prediction").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Classified test set :
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85426|
|  1.0|  394|
+-----+-----+

Prediction :
+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|85505|
|       1.0|  315|
+----------+-----+



In [53]:
println(s"Matrice de confusion :")
predictionsMLP.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice de confusion :
+----------+-----+-----+
|prediction|label|count|
+----------+-----+-----+
|       0.0|  0.0|85401|
|       1.0|  0.0|   25|
|       0.0|  1.0|  104|
|       1.0|  1.0|  290|
+----------+-----+-----+



In [54]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val accuracy = evaluator1.evaluate(predictionsMLP)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsMLP))
println("Accuracy = " + evaluator1.evaluate(predictionsMLP))
println("Test Error = " + (1.0 - accuracy))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluator1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_dd4048e87b17
areaUnderROC: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_4ce967996f9d
accuracy: Double = 0.9984968538802144
Area Under ROC Curve = 0.8678739790938481
Accuracy = 0.9984968538802144
Test Error = 0.0015031461197856366


In [55]:
// One Vs Rest Classifier using Logistic Regression classifier
val classifier = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setTol(1E-6).setFitIntercept(true) 
val t = System.nanoTime

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

classifier: org.apache.spark.ml.classification.LogisticRegression = logreg_5e19b714ead7
t: Long = 635029526951555


In [56]:
// train the multiclass model.
val ovr = new OneVsRest().setClassifier(classifier)
val Modelovr = ovr.fit(trainingData)
val durationtrain = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationtrain secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ovr: org.apache.spark.ml.classification.OneVsRest = oneVsRest_f0608d461a39
Modelovr: org.apache.spark.ml.classification.OneVsRestModel = oneVsRest_f0608d461a39
durationtrain: Double = 10.744430794

initial model training finished.
Training process takes 10.744430794 secs


In [ ]:
val s = System.nanoTime
val predictionsOVR = Modelovr.transform(validationData)
val durationprediction = (System.nanoTime - s) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationprediction secs")
predictionsOVR.select("prediction", "label", "features").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
println(s"Classified test set :")
validationData.groupBy("Class").count().show()
println(s"Prediction :")
predictionsOVR.groupBy("prediction").count().show()

In [ ]:
println(s"Matrice de confusion :")
predictionsOVR.select("prediction", "label").groupBy("prediction", "label").count().orderBy("label", "prediction").show()

In [ ]:
val evaluator1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("accuracy")
val evaluator2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedPrecision")
val evaluator3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("weightedRecall")
val evaluator4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("f1")
val areaUnderROC = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderROC")
val areaUnderPR = new BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("label").setMetricName("areaUnderPR")
val accuracy = evaluator1.evaluate(predictionsOVR)
println("Area Under ROC Curve = " + areaUnderROC.evaluate(predictionsOVR))
println("Area Under the Precision-Recall Curve = "  + areaUnderPR.evaluate(predictionsOVR))
println("Accuracy = " + evaluator1.evaluate(predictionsOVR))
println("Precision = " + evaluator2.evaluate(predictionsOVR))
println("Recall = " + evaluator3.evaluate(predictionsOVR))
println("F1 = " + evaluator4.evaluate(predictionsOVR))
println("Test Error = " + (1.0 - accuracy))